In [0]:
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import functions as sf

In [0]:
# Crafting Skill activities
schema_crafting = "del string, name string, level int, xp int, num_needed int, input string, input_cost string, output_price string, profit_loss string, gp_xp_ratio string, members string"
# data is raw data from https://oldschool.runescape.wiki/w/Calculator:Crafting

# TODO replace with reference to file in the landing
data_crafting = [
  ["","Air battlestaff",66,137.5,1,"1 ×  Battlestaff\n1 ×  Air orb","9,220","8,619","−601","−4.37","Members"  ],
  ["","Amethyst arrowtips",85,60,1,"1 ×  Amethyst","3,324","3,330","6","0.10","Members"  ],
  ["","Amethyst bolt tips",83,60,1,"1 ×  Amethyst","3,324","2,520","−804","−13.40","Members"  ],
  ["","Amethyst dart tip",89,60,1,"1 ×  Amethyst","3,324","3,288","−36","−0.60","Members"  ],
  ["","Amethyst javelin tips",87,60,1,"1 ×  Amethyst","3,324","3,235","−89","−1.48","Members"  ],
  ["","Ball of wool",1,2.5,1,"1 ×  Wool","27","58","31","12.40","Free-to-play"  ],
  ["","Basket",36,56,1,"6 ×  Willow branch","10,770","2","−10,768","−192.29","Members"  ],
  ["","Beer glass",1,17.5,1,"1 ×  Molten glass","96","25","−71","−4.06","Members"  ],
  ["","Bird house (item)",5,15,1,"1 ×  Clockwork\n1 ×  Logs","962","525","−437","−29.13","Members"  ],
  ["","Black d'hide body",84,258,1,"3 ×  Black dragon leather\n0.2 ×  Thread","11,639","7,035","−4,604","−17.84","Members"  ],
  ["","Black d'hide chaps",82,172,1,"2 ×  Black dragon leather\n0.2 ×  Thread","7,760","3,764","−3,996","−23.23","Members"  ],
  ["","Black d'hide shield",83,172,1,"2 ×  Black dragon leather\n1 ×  Redwood shield\n15 ×  Rune nails","20,271","18,029","−2,242","−13.03","Members"  ],
  ["","Black d'hide vambraces",79,86,1,"1 ×  Black dragon leather\n0.2 ×  Thread","3,881","2,079","−1,802","−20.95","Members"  ],
  ["","Blood'n'tar snelm",15,32.5,1,"1 ×  Blamish red shell (round)","316","383","67","2.06","Members"  ],
  ["","Blue d'hide body",71,210,1,"3 ×  Blue dragon leather\n0.2 ×  Thread","6,620","4,902","−1,718","−8.18","Members"  ],
  ["","Blue d'hide chaps",68,140,1,"2 ×  Blue dragon leather\n0.2 ×  Thread","4,414","2,031","−2,383","−17.02","Members"  ],
  ["","Blue d'hide shield",69,140,1,"2 ×  Blue dragon leather\n1 ×  Yew shield\n15 ×  Mithril nails","5,857","4,751","−1,106","−7.90","Members"  ],
  ["","Blue d'hide vambraces",66,70,1,"1 ×  Blue dragon leather\n0.2 ×  Thread","2,208","1,372","−836","−11.94","Members"  ],
  ["","Bow string",10,15,1,"1 ×  Flax","2","89","87","5.80","Members"  ],
  ["","Bowl",8,15,1,"1 ×  Unfired bowl","216","18","−198","−13.20","Free-to-play"  ],
  ["","Bowl (full)",8,33,1,"1 ×  Soft clay","125","18","−107","−3.24","Free-to-play"  ],
  ["","Broken bark snelm",15,32.5,1,"1 ×  Blamish bark shell","1,135","71","−1,064","−32.74","Members"  ],
  ["","Broodoo shield (combat)",35,100,1,"2 ×  Snakeskin\n1 ×  Tribal mask (combat)\n8 ×  Steel nails","2,166","5,382","3,216","32.16","Members"  ],
  ["","Broodoo shield (disease)",35,100,1,"2 ×  Snakeskin\n1 ×  Tribal mask (disease)\n8 ×  Steel nails","2,008","6,077","4,069","40.69","Members"  ],
  ["","Broodoo shield (poison)",35,100,1,"2 ×  Snakeskin\n1 ×  Tribal mask (poison)\n8 ×  Steel nails","2,626","5,919","3,293","32.93","Members"  ],
  ["","Bruise blue snelm",15,32.5,1,"1 ×  Blamish blue shell (round)","1,141","1,232","91","2.80","Members"  ],
  ["","Coif",38,37,1,"1 ×  Leather\n0.2 ×  Thread","239","17","−222","−6","Members"  ],
  ["","Crab claw",15,32.5,1,"1 ×  Fresh crab claw","0","0","0","0","Members"  ],
  ["","Crab helmet",15,32.5,1,"1 ×  Fresh crab shell","0","0","0","0","Members"  ],
  ["","Crossbow string (roots)",10,15,1,"1 ×  Oak roots","132","336","204","13.60","Members"  ],
  ["","Crossbow string (sinew)",10,15,1,"1 ×  Sinew","138","336","198","13.20","Members"  ],
  ["","Diamond",43,107.5,1,"1 ×  Uncut diamond","2,979","1,760","−1,219","−11.34","Free-to-play"  ],
  ["","Diamond amulet (u)",70,100,1,"1 ×  Diamond\n1 ×  Gold bar","1,902","1,913","11","0.11","Free-to-play"  ],
  ["","Diamond bracelet",58,95,1,"1 ×  Diamond\n1 ×  Gold bar","1,902","1,950","48","0.51","Members"  ],
  ["","Diamond necklace",56,90,1,"1 ×  Diamond\n1 ×  Gold bar","1,902","2,012","110","1.22","Free-to-play"  ],
  ["","Diamond ring",43,85,1,"1 ×  Diamond\n1 ×  Gold bar","1,902","1,925","23","0.27","Free-to-play"  ],
  ["","Dragon necklace",72,105,1,"1 ×  Dragonstone\n1 ×  Gold bar","11,337","11,185","−152","−1.45","Members"  ],
  ["","Dragonstone",55,137.5,1,"1 ×  Uncut dragonstone","17,125","11,006","−6,119","−44.50","Members"  ],
  ["","Dragonstone amulet (u)",80,150,1,"1 ×  Dragonstone\n1 ×  Gold bar","11,337","11,142","−195","−1.30","Members"  ],
  ["","Dragonstone bracelet",74,110,1,"1 ×  Dragonstone\n1 ×  Gold bar","11,337","11,274","−63","−0.57","Members"  ],
  ["","Dragonstone ring",55,100,1,"1 ×  Dragonstone\n1 ×  Gold bar","11,337","11,139","−198","−1.98","Members"  ],
  ["","Drift net",26,55,1,"2 ×  Jute fibre","584","1,104","520","9.45","Members"  ],
  ["","Earth battlestaff",58,112.5,1,"1 ×  Battlestaff\n1 ×  Earth orb","8,921","8,611","−310","−2.76","Members"  ],
  ["","Emerald",27,67.5,1,"1 ×  Uncut emerald","593","513","−80","−1.19","Free-to-play"  ],
  ["","Emerald amulet (u)",31,70,1,"1 ×  Emerald\n1 ×  Gold bar","630","632","2","0.03","Free-to-play"  ],
  ["","Emerald bracelet",30,65,1,"1 ×  Emerald\n1 ×  Gold bar","630","729","99","1.52","Members"  ],
  ["","Emerald necklace",29,65,1,"1 ×  Emerald\n1 ×  Gold bar","630","691","61","0.94","Free-to-play"  ],
  ["","Emerald ring",27,55,1,"1 ×  Emerald\n1 ×  Gold bar","630","664","34","0.62","Free-to-play"  ],
  ["","Empty candle lantern",4,19,1,"1 ×  Molten glass","96","4","−92","−4.84","Members"  ],
  ["","Empty fishbowl",42,42.5,1,"1 ×  Molten glass","96","13","−83","−1.95","Members"  ],
  ["","Empty light orb",87,70,1,"1 ×  Molten glass","96","0","−96","−1.37","Members"  ],
  ["","Empty oil lamp",12,25,1,"1 ×  Molten glass","96","3","−93","−3.72","Members"  ],
  ["","Empty plant pot",19,17.5,1,"1 ×  Unfired plant pot","119","6","−113","−6.46","Members"  ],
  ["","Empty plant pot (full)",19,37.5,1,"1 ×  Soft clay","125","6","−119","−3.17","Members"  ],
  ["","Empty sack",21,38,1,"4 ×  Jute fibre","1,168","3","−1,165","−30.66","Members"  ],
  ["","Fire battlestaff",62,125,1,"1 ×  Battlestaff\n1 ×  Fire orb","9,151","8,623","−528","−4.22","Members"  ],
  ["","Gold amulet (u)",8,30,1,"1 ×  Gold bar","107","135","28","0.93","Free-to-play"  ],
  ["","Gold bracelet",7,25,1,"1 ×  Gold bar","107","245","138","5.52","Members"  ],
  ["","Gold necklace",6,20,1,"1 ×  Gold bar","107","131","24","1.20","Free-to-play"  ],
  ["","Gold ring",5,15,1,"1 ×  Gold bar","107","123","16","1.07","Free-to-play"  ],
  ["","Gold tiara",42,35,1,"1 ×  Gold bar","107","33","−74","−2.11","Members"  ],
  ["","Green d'hide body",63,186,1,"3 ×  Green dragon leather\n0.2 ×  Thread","6,296","4,314","−1,982","−10.66","Members"  ],
  ["","Green d'hide chaps",60,124,1,"2 ×  Green dragon leather\n0.2 ×  Thread","4,198","2,179","−2,019","−16.28","Members"  ],
  ["","Green d'hide shield",62,124,1,"2 ×  Green dragon leather\n1 ×  Maple shield\n15 ×  Steel nails","4,604","3,179","−1,425","−11.49","Members"  ],
  ["","Green d'hide vambraces",57,62,1,"1 ×  Green dragon leather\n0.2 ×  Thread","2,100","1,400","−700","−11.29","Members"  ],
  ["","Hard leather shield",41,70,1,"2 ×  Hard leather\n1 ×  Oak shield\n15 ×  Bronze nails","638","167","−471","−6.73","Members"  ],
  ["","Hardleather body",28,35,1,"1 ×  Hard leather\n0.2 ×  Thread","269","10","−259","−7.40","Free-to-play"  ],
  ["","Installing light orbs",52,500,1,"1 ×  Light orb","551","0","−551","−1.10","Members"  ],
  ["","Jade",13,20,1,"1 ×  Uncut jade","92","276","184","9.20","Members"  ],
  ["","Jade amulet (u)",34,70,1,"1 ×  Jade\n1 ×  Silver bar","383","805","422","6.03","Members"  ],
  ["","Jade bracelet",29,60,1,"1 ×  Jade\n1 ×  Silver bar","383","654","271","4.52","Members"  ],
  ["","Jade necklace",25,54,1,"1 ×  Jade\n1 ×  Silver bar","383","889","506","9.37","Members"  ],
  ["","Jade ring",13,32,1,"1 ×  Jade\n1 ×  Silver bar","383","688","305","9.53","Members"  ],
  ["","Lantern lens",49,55,1,"1 ×  Molten glass","96","12","−84","−1.53","Members"  ],
  ["","Leather body",14,25,1,"1 ×  Leather\n0.2 ×  Thread","239","8","−231","−9.24","Free-to-play"  ],
  ["","Leather boots",7,16.2,1,"1 ×  Leather\n0.2 ×  Thread","239","24","−215","−13.27","Free-to-play"  ],
  ["","Leather chaps",18,27,1,"1 ×  Leather\n0.2 ×  Thread","239","5","−234","−8.67","Free-to-play"  ],
  ["","Leather cowl",9,18.5,1,"1 ×  Leather\n0.2 ×  Thread","239","1","−238","−12.86","Free-to-play"  ],
  ["","Leather gloves",1,13.8,1,"1 ×  Leather\n0.2 ×  Thread","239","1","−238","−17.25","Free-to-play"  ],
  ["","Leather vambraces",11,22,1,"1 ×  Leather\n0.2 ×  Thread","239","4","−235","−10.68","Free-to-play"  ],
  ["","Light orb",87,104,1,"1 ×  Empty light orb\n1 ×  Cave goblin wire","1,190","540","−650","−6.25","Members"  ],
  ["","Magic bird house",75,50,1,"1 ×  Clockwork\n1 ×  Magic logs","1,954","2,160","206","4.12","Members"  ],
  ["","Magic string",19,30,1,"1 ×  Magic roots","2,845","1,593","−1,252","−41.73","Members"  ],
  ["","Mahogany bird house",50,40,1,"1 ×  Clockwork\n1 ×  Mahogany logs","1,195","1,352","157","3.93","Members"  ],
  ["","Maple bird house",45,35,1,"1 ×  Clockwork\n1 ×  Maple logs","944","1,505","561","16.03","Members"  ],
  ["","Molten glass",1,20,1,"1 ×  Bucket of sand\n1 ×  Soda ash","33","95","62","3.10","Members"  ],
  ["","Molten glass (Superglass Make with Staff)",61,180,1,"18 ×  Bucket of sand\n3 ×  Giant seaweed\n2 ×  Astral rune","1,022","2,223","1,201","6.67","Members"  ],
  ["","Molten glass (Superglass Make)",61,180,1,"18 ×  Bucket of sand\n3 ×  Giant seaweed\n2 ×  Astral rune\n6 ×  Fire rune\n10 ×  Air rune","1,102","2,223","1,121","6.23","Members"  ],
  ["","Myre snelm",15,32.5,1,"1 ×  Blamish myre shell (round)","332","75","−257","−7.91","Members"  ],
  ["","Oak bird house",15,20,1,"1 ×  Clockwork\n1 ×  Oak logs","955","502","−453","−22.65","Members"  ],
  ["","Ochre snelm",15,32.5,1,"1 ×  Blamish ochre shell (round)","166","45","−121","−3.72","Members"  ],
  ["","Oil lantern",26,50,1,"1 ×  Empty oil lamp\n1 ×  Oil lantern frame","79","543","464","9.28","Members"  ],
  ["","Onyx",67,167.5,1,"1 ×  Uncut onyx","2,139,457","1,976,112","−163,345","−975.19","Members"  ],
  ["","Onyx amulet (u)",90,165,1,"1 ×  Onyx\n1 ×  Gold bar","2,016,547","2,261,643","245,096","1,485.43","Members"  ],
  ["","Onyx bracelet",84,125,1,"1 ×  Onyx\n1 ×  Gold bar","2,016,547","2,220,028","203,481","1,627.85","Members"  ],
  ["","Onyx necklace",82,120,1,"1 ×  Onyx\n1 ×  Gold bar","2,016,547","2,186,252","169,705","1,414.21","Members"  ],
  ["","Onyx ring",67,115,1,"1 ×  Onyx\n1 ×  Gold bar","2,016,547","1,789,624","−226,923","−1,973.24","Members"  ],
  ["","Opal",1,15,1,"1 ×  Uncut opal","90","248","158","10.53","Members"  ],
  ["","Opal amulet (u)",27,55,1,"1 ×  Opal\n1 ×  Silver bar","355","515","160","2.91","Members"  ],
  ["","Opal bracelet",22,45,1,"1 ×  Opal\n1 ×  Silver bar","355","728","373","8.29","Members"  ],
  ["","Opal necklace",16,35,1,"1 ×  Opal\n1 ×  Silver bar","355","806","451","12.89","Members"  ],
  ["","Opal ring",1,10,1,"1 ×  Opal\n1 ×  Silver bar","355","794","439","43.90","Members"  ],
  ["","Pie dish",7,10,1,"1 ×  Unfired pie dish","153","24","−129","−12.90","Free-to-play"  ],
  ["","Pie dish (full)",7,25,1,"1 ×  Soft clay","125","24","−101","−4.04","Free-to-play"  ],
  ["","Pointed blood'n'tar snelm",15,32.5,1,"1 ×  Blamish red shell (pointed)","700","1,302","602","18.52","Members"  ],
  ["","Pointed bruise blue snelm",15,32.5,1,"1 ×  Blamish blue shell (pointed)","2,995","3,842","847","26.06","Members"  ],
  ["","Pointed myre snelm",15,32.5,1,"1 ×  Blamish myre shell (pointed)","277","147","−130","−4","Members"  ],
  ["","Pointed ochre snelm",15,32.5,1,"1 ×  Blamish ochre shell (pointed)","273","575","302","9.29","Members"  ],
  ["","Pot",1,6.3,1,"1 ×  Unfired pot","134","30","−104","−16.51","Free-to-play"  ],
  ["","Pot (full)",1,12.6,1,"1 ×  Soft clay","125","30","−95","−7.54","Free-to-play"  ],
  ["","Pot lid",25,20,1,"1 ×  Unfired pot lid","108","288","180","9","Members"  ],
  ["","Pot lid (full)",25,40,1,"1 ×  Soft clay","125","288","163","4.08","Members"  ],
  ["","Red d'hide body",77,234,1,"3 ×  Red dragon leather\n0.2 ×  Thread","8,579","5,851","−2,728","−11.66","Members"  ],
  ["","Red d'hide chaps",75,156,1,"2 ×  Red dragon leather\n0.2 ×  Thread","5,720","2,596","−3,124","−20.03","Members"  ],
  ["","Red d'hide shield",76,156,1,"2 ×  Red dragon leather\n1 ×  Magic shield\n15 ×  Adamant nails","6,382","6,036","−346","−2.22","Members"  ],
  ["","Red d'hide vambraces",73,78,1,"1 ×  Red dragon leather\n0.2 ×  Thread","2,861","1,590","−1,271","−16.29","Members"  ],
  ["","Red topaz",16,25,1,"1 ×  Uncut red topaz","2,910","3,409","499","19.96","Members"  ],
  ["","Redwood bird house",90,55,1,"1 ×  Clockwork\n1 ×  Redwood logs","1,865","2,617","752","13.67","Members"  ],
  ["","Rope",30,25,1,"1 ×  Hair","293","49","−244","−9.76","Members"  ],
  ["","Ruby",34,85,1,"1 ×  Uncut ruby","1,253","922","−331","−3.89","Free-to-play"  ],
  ["","Ruby amulet (u)",50,85,1,"1 ×  Ruby\n1 ×  Gold bar","1,047","1,078","31","0.36","Free-to-play"  ],
  ["","Ruby bracelet",42,80,1,"1 ×  Ruby\n1 ×  Gold bar","1,047","1,129","82","1.02","Members"  ],
  ["","Ruby necklace",40,75,1,"1 ×  Ruby\n1 ×  Gold bar","1,047","1,145","98","1.31","Free-to-play"  ],
  ["","Ruby ring",34,70,1,"1 ×  Ruby\n1 ×  Gold bar","1,047","1,070","23","0.33","Free-to-play"  ],
  ["","Sapphire",20,50,1,"1 ×  Uncut sapphire","335","316","−19","−0.38","Free-to-play"  ],
  ["","Sapphire amulet (u)",24,65,1,"1 ×  Sapphire\n1 ×  Gold bar","429","426","−3","−0.05","Free-to-play"  ],
  ["","Sapphire bracelet",23,60,1,"1 ×  Sapphire\n1 ×  Gold bar","429","464","35","0.58","Members"  ],
  ["","Sapphire necklace",22,55,1,"1 ×  Sapphire\n1 ×  Gold bar","429","481","52","0.95","Free-to-play"  ],
  ["","Sapphire ring",20,40,1,"1 ×  Sapphire\n1 ×  Gold bar","429","472","43","1.08","Free-to-play"  ],
  ["","Silver bolts (unf)",21,50,1,"1 ×  Silver bar","102","130","28","0.56","Members"  ],
  ["","Slayer ring",75,15,1,"1 ×  Enchanted gem\n1 ×  Gold bar","108","0","−108","−7.20","Members"  ],
  ["","Snakeskin bandana",48,45,1,"5 ×  Snakeskin\n0.2 ×  Thread","107","379","272","6.04","Members"  ],
  ["","Snakeskin body",53,55,1,"15 ×  Snakeskin\n0.2 ×  Thread","317","922","605","11","Members"  ],
  ["","Snakeskin boots",45,30,1,"6 ×  Snakeskin\n0.2 ×  Thread","128","345","217","7.23","Members"  ],
  ["","Snakeskin chaps",51,50,1,"12 ×  Snakeskin\n0.2 ×  Thread","254","629","375","7.50","Members"  ],
  ["","Snakeskin shield",56,100,1,"2 ×  Snakeskin\n1 ×  Willow shield\n15 ×  Iron nails","134","272","138","1.38","Members"  ],
  ["","Snakeskin vambraces",47,35,1,"8 ×  Snakeskin\n0.2 ×  Thread","170","274","104","2.97","Members"  ],
  ["","Strip of cloth",10,12,1,"4 ×  Ball of wool","236","0","−236","−19.67","Members"  ],
  ["","Studded body",41,40,1,"1 ×  Leather body\n1 ×  Steel studs","214","352","138","3.45","Members"  ],
  ["","Studded chaps",44,42,1,"1 ×  Leather chaps\n1 ×  Steel studs","211","291","80","1.90","Members"  ],
  ["","Teak bird house",35,30,1,"1 ×  Clockwork\n1 ×  Teak logs","1,050","1,468","418","13.93","Members"  ],
  ["","Tiara",23,52.5,1,"1 ×  Silver bar","102","98","−4","−0.08","Free-to-play"  ],
  ["","Topaz amulet (u)",45,80,1,"1 ×  Red topaz\n1 ×  Silver bar","3,580","4,906","1,326","16.58","Members"  ],
  ["","Topaz bracelet",38,75,1,"1 ×  Red topaz\n1 ×  Silver bar","3,580","3,911","331","4.41","Members"  ],
  ["","Topaz necklace",32,70,1,"1 ×  Red topaz\n1 ×  Silver bar","3,580","1,452","−2,128","−30.40","Members"  ],
  ["","Topaz ring",16,35,1,"1 ×  Red topaz\n1 ×  Silver bar","3,580","4,279","699","19.97","Members"  ],
  ["","Unfired bowl",8,18,1,"1 ×  Soft clay","125","212","87","4.83","Free-to-play"  ],
  ["","Unfired pie dish",7,15,1,"1 ×  Soft clay","125","150","25","1.67","Free-to-play"  ],
  ["","Unfired plant pot",19,20,1,"1 ×  Soft clay","125","117","−8","−0.40","Members"  ],
  ["","Unfired pot",1,6.3,1,"1 ×  Soft clay","125","132","7","1.11","Free-to-play"  ],
  ["","Unfired pot lid",25,20,1,"1 ×  Soft clay","125","106","−19","−0.95","Members"  ],
  ["","Unpowered orb",46,52.5,1,"1 ×  Molten glass","96","52","−44","−0.84","Members"  ],
  ["","Unstrung emblem",17,50,1,"1 ×  Silver bar","102","151","49","0.98","Members"  ],
  ["","Unstrung symbol",16,50,1,"1 ×  Silver bar","102","88","−14","−0.28","Free-to-play"  ],
  ["","Vial",33,35,1,"1 ×  Molten glass","96","2","−94","−2.69","Members"  ],
  ["","Water battlestaff",54,100,1,"1 ×  Battlestaff\n1 ×  Water orb","8,836","8,627","−209","−2.09","Members"  ],
  ["","Willow bird house",25,25,1,"1 ×  Clockwork\n1 ×  Willow logs","954","1,311","357","14.28","Members"  ],
  ["","Xerician hat",14,66,1,"3 ×  Xerician fabric\n0.2 ×  Thread","7,469","7,279","−190","−2.88","Members"  ],
  ["","Xerician robe",17,88,1,"4 ×  Xerician fabric\n0.2 ×  Thread","9,958","9,457","−501","−5.69","Members"  ],
  ["","Xerician top",22,110,1,"5 ×  Xerician fabric\n0.2 ×  Thread","12,447","9,752","−2,695","−24.50","Members"  ],
  ["","Yak-hide legs",43,32,1,"1 ×  Cured yak-hide","1,006","1,427","421","13.16","Members"  ],
  ["","Yak-hide top",46,32,1,"2 ×  Cured yak-hide","2,012","0","−2,012","−62.87","Members"  ],
  ["","Yew bird house",60,45,1,"1 ×  Clockwork\n1 ×  Yew logs","1,062","1,386","324","7.20","Members"  ],
  ["","Zenyte",89,200,1,"1 ×  Uncut zenyte","16,871,364","17,115,141","243,777","1,218.89","Members"  ],
  ["","Zenyte amulet (u)",98,200,1,"1 ×  Zenyte\n1 ×  Gold bar","17,464,536","15,403,914","−2,060,622","−10,303.11","Members"  ],
  ["","Zenyte bracelet",95,180,1,"1 ×  Zenyte\n1 ×  Gold bar","17,464,536","20,673,334","3,208,798","17,826.66","Members"  ],
  ["","Zenyte necklace",92,165,1,"1 ×  Zenyte\n1 ×  Gold bar","17,464,536","20,484,088","3,019,552","18,300.32","Members"  ],
  ["","Zenyte ring",89,150,1,"1 ×  Zenyte\n1 ×  Gold bar","17,464,536","20,410,105","2,945,569","19,637.13","Members"  ]
]

df_crafting = spark.createDataFrame(data_crafting, schema=schema_crafting)
# select only columns that we care about
df_crafting = df_crafting.select("name", "xp", "input")

# need to split the input column
df = df_crafting.select('*', sf.split(df_crafting.input, '[\n]').alias("split_input"))

# df = df.select('*', sf.split(df.split_input, '[x]')).display()

In [0]:
df.display()

In [0]:
df2 = df.select('name', sf.explode('split_input').alias("input"))
df2.display()

In [0]:
# split string by the multiplcation symbol '×'
df3 = df2.select('name', sf.split(df2.input, '[×]').alias("input"))
df3.display()

In [0]:
df4 = df3.select('name', sf.get(df3.input,0).alias("input_qty"), sf.get(df3.input,1).alias("input_name"))
df4.display()